In [1]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

In [2]:
# Load the data
base_dir = "C:/Users/natda/Desktop/NatDave/Academics/PhD_NU/RESEARCH/BlueBikes/"
df = pd.read_csv(base_dir + "all_trips.csv")

# Replace 'electric_bike' with 'electric' and 'classic_bike' with 'traditional'
df['bike_type'] = df['bike_type'] \
                        .replace({'electric_bike': 'electric',
                                  'classic_bike': 'traditional'})

# Rename the columns
df = df.rename(columns={"member_casual": "rider_type", "rideable_type": "bike_type"})

In [3]:
print(f"Data Shape: {df.shape}")

Data Shape: (4921170, 13)


In [4]:
df.dtypes

ride_id                object
bike_type              object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
rider_type             object
dtype: object

In [5]:
# Convert started_at and ended_at columns to datetime
df["started_at"] = pd.to_datetime(df["started_at"], errors='coerce')
df["ended_at"] = pd.to_datetime(df["ended_at"], errors='coerce')

# Check the data types after conversion
df[["started_at", "ended_at"]].dtypes

started_at    datetime64[ns]
ended_at      datetime64[ns]
dtype: object

In [6]:
# Function to check for fractional seconds
def check_fractional_seconds(df, *features):
    fractional_seconds = df[df[features[0]].astype(str).str.contains(r"\.\d+")]

    if fractional_seconds.empty:
        print("No rows with fractional seconds found.")
    else:
        print(fractional_seconds[[*features]].head())

# Check for fractional seconds
check_fractional_seconds(df, 'started_at', 'ended_at')

No rows with fractional seconds found.


In [7]:
# Identify rows where conversion to datetime failed
invalid_dates = df[df["started_at"].isna() | df["ended_at"].isna()]

if invalid_dates.empty:
    print("No invalid dates found.")
else:
    # Display the first few rows with invalid dates
    print("Invalid date entries (showing first 5 rows):")
    print(invalid_dates[["started_at", "ended_at"]].head())
    
    # Count the number of invalid date entries
    num_invalid_dates = invalid_dates.shape[0]
    print(f"\nNumber of rows with invalid dates: {num_invalid_dates}.")

No invalid dates found.


In [8]:
invalid_dates

,ride_id,bike_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,rider_type


In [9]:
# Missing values
missing_values = df.isnull().sum()
missing_values

ride_id                  0
bike_type                0
started_at               0
ended_at                 0
start_station_name    1443
start_station_id      1443
end_station_name      8748
end_station_id        9151
start_lat                0
start_lng                0
end_lat               3962
end_lng               3962
rider_type               0
dtype: int64

In [10]:
# Remove rows with any missing values
df = df.dropna()

In [11]:
def validate_station_mappings(dataframe):
    """
    Validate Station Name-to-ID mappings for both start and end stations.

    This function checks for inconsistencies between station names and station IDs by identifying
    cases where a station name is associated with multiple station IDs or where a station ID is
    associated with multiple station names. The results are printed to the console.

    Parameters:
        dataframe (pd.DataFrame): The DataFrame containing bike trip data with station names and IDs.
    """

    # Create dictionaries to map start station names to IDs and start station IDs to names
    start_name_to_ids = dataframe.groupby("start_station_name")["start_station_id"].unique().to_dict()
    start_id_to_names = dataframe.groupby("start_station_id")["start_station_name"].unique().to_dict()

    # Create dictionaries to map end station names to IDs and end station IDs to names
    end_name_to_ids = dataframe.groupby("end_station_name")["end_station_id"].unique().to_dict()
    end_id_to_names = dataframe.groupby("end_station_id")["end_station_name"].unique().to_dict()

    # Check for mismatches in start stations (name to multiple IDs)
    print("Start Station Names Mapping to Multiple IDs:")
    if any(len(ids) > 1 for ids in start_name_to_ids.values()):
        for name, ids in start_name_to_ids.items():
            if len(ids) > 1:
                print(f"{name}: {list(ids)} (Instances: {len(ids)})")
    else:
        print("nil")

    # Check for mismatches in start stations (ID to multiple names)
    print("\nStart Station IDs Mapping to Multiple Names:")
    if any(len(names) > 1 for names in start_id_to_names.values()):
        for station_id, names in start_id_to_names.items():
            if len(names) > 1:
                print(f"{station_id}: {list(names)} (Instances: {len(names)})")
    else:
        print("nil")

    # Check for mismatches in end stations (name to multiple IDs)
    print("\nEnd Station Names Mapping to Multiple IDs:")
    if any(len(ids) > 1 for ids in end_name_to_ids.values()):
        for name, ids in end_name_to_ids.items():
            if len(ids) > 1:
                print(f"{name}: {list(ids)} (Instances: {len(ids)})")
    else:
        print("nil")

    # Check for mismatches in end stations (ID to multiple names)
    print("\nEnd Station IDs Mapping to Multiple Names:")
    if any(len(names) > 1 for names in end_id_to_names.values()):
        for station_id, names in end_id_to_names.items():
            if len(names) > 1:
                print(f"{station_id}: {list(names)} (Instances: {len(names)})")
    else:
        print("nil")

validate_station_mappings(df)

Start Station Names Mapping to Multiple IDs:
Somerville Hospital: ['S32020', 'S32052'] (Instances: 2)
Tremont St at Court St: ['A32046', 'A32058'] (Instances: 2)

Start Station IDs Mapping to Multiple Names:
A32024: ['Staniford at Merrimac', 'Staniford St at Merrimac St'] (Instances: 2)
A32046: ['Tremont St at Court St', 'Canal St. at Causeway St.', 'Canal St at Causeway St'] (Instances: 3)
A32058: ['Tremont St. at Court St.', 'Tremont St at Court St'] (Instances: 2)
B32038: ['Chestnut Hill Ave. at Ledgemere Road', 'Chestnut Hill Ave at Ledgemere Rd'] (Instances: 2)
C32054: ['Shawmut Ave. at Herald St.', 'Shawmut Ave at Herald St'] (Instances: 2)
C32109: ['Centre St. at Allandale St.', 'Centre St at Allandale St'] (Instances: 2)
E32003: ['Hyde Square - Barbara St at Centre St', 'Hyde Square - Centre St at Perkins St'] (Instances: 2)
L32007: ['Swan Pl. at Minuteman Bikeway', 'Swan Place at Minuteman Bikeway'] (Instances: 2)
L32010: ['Medford St. at Warren St.', 'Medford St at Warren St'

In [12]:
# Delete trips starting or ending at station ID "S32020" (this station does not exist)
start_S32020 = df["start_station_id"].value_counts().get("S32020", 0)
end_S32020 = df["end_station_id"].value_counts().get("S32020", 0)

S32020_mismatch = start_S32020 + end_S32020
print(f"{S32020_mismatch} of the trips start or end at station S32020.")

df = df[~((df["start_station_id"] == "S32020") |
                            (df["end_station_id"] == "S32020"))]

138 of the trips start or end at station S32020.


In [13]:
# Count instances of "A32046" under start and end station IDs for the name "Tremont St at Court St"
count_start = df[(df["start_station_id"] == "A32046") &
                          (df["start_station_name"] == "Tremont St at Court St")].shape[0]

count_end = df[(df["end_station_id"] == "A32046") &
                        (df["end_station_name"] == "Tremont St at Court St")].shape[0]

# Total instances
total_count = count_start + count_end
print(f"Total instances of station ID A32046 mismatched with Tremont St at Court St: {total_count}")

# Update start_station_name and end_station_name for "A32046" instances
# Station ID A32046 is for "Canal St at Causeway St" not "Tremont St at Court St"

df.loc[(df["start_station_id"] == "A32046") &
                (df["start_station_name"] == "Tremont St at Court St"),
                "start_station_name"] = "Canal St at Causeway St"

df.loc[(df["end_station_id"] == "A32046") &
                (df["end_station_name"] == "Tremont St at Court St"),
                "end_station_name"] = "Canal St at Causeway St"

Total instances of station ID A32046 mismatched with Tremont St at Court St: 34


In [14]:
def update_station_names(dataframe, old_name, new_name):
    """
    Update station names in both start and end station columns of the DataFrame.

    This function replaces occurrences of the specified old station name with the new station name
    in both "start_station_name" and "end_station_name" columns of the DataFrame.

    Parameters:
        dataframe (pd.DataFrame): The DataFrame containing bike trip data with station names.
        old_name (str): The old station name to be replaced.
        new_name (str): The new station name to replace with.
    """
    # Update both start and end station names
    dataframe.loc[dataframe["start_station_name"] == old_name, "start_station_name"] = new_name
    dataframe.loc[dataframe["end_station_name"] == old_name, "end_station_name"] = new_name

# List of changes to be made
changes = [
    ["Canal St. at Causeway St.", "Canal St at Causeway St"],
    ["Tremont St. at Court St.", "Tremont St at Court St"],
    ["Chestnut Hill Ave. at Ledgemere Road", "Chestnut Hill Ave at Ledgemere Rd"],
    ["Centre St. at Allandale St.", "Centre St at Allandale St"],
    ["Hyde Square - Barbara St at Centre St", "Hyde Square - Centre St at Perkins St"],
    ["Swan Pl. at Minuteman Bikeway", "Swan Place at Minuteman Bikeway"],
    ["CambridgeSide Galleria - CambridgeSide PL at Land Blvd", "Cambridgeside Pl at Land Blvd"],
    ["Summer St at Quincy St", "Somerville Hospital"],
    ["Everett Square (Broadway at Chelsea St)", "Everett Square (Broadway at Norwood St)"],
    ["Damrell st at Old Colony Ave", "Damrell St at Old Colony Ave"],
    ["Staniford at Merrimac", "Staniford St at Merrimac St"],
    ["Shawmut Ave. at Herald St.", "Shawmut Ave at Herald St"],
    ["Medford St. at Warren St.", "Medford St at Warren St"]
]

# Applying the changes to the DataFrame
for old_name, new_name in changes:
    update_station_names(df, old_name, new_name)

# Confirmation message
print("Station names updated successfully.")

Station names updated successfully.


In [15]:
# Check for mismatch again
validate_station_mappings(df)

Start Station Names Mapping to Multiple IDs:
nil

Start Station IDs Mapping to Multiple Names:
nil

End Station Names Mapping to Multiple IDs:
nil

End Station IDs Mapping to Multiple Names:
A32046: ['Canal St\xa0at\xa0Causeway\xa0St', 'Canal St at Causeway St'] (Instances: 2)


In [16]:
"""
The \xa0 character is the Unicode representation for a non-breaking space (NBSP). This character
is different from a regular space ( " " , Unicode U+0020 ) although they appear the same.
"""

# Standardize station names for A32046
standard_name = "Canal St at Causeway St"

# Identify and replace the inconsistent names
df.loc[
    (df["end_station_id"] == "A32046") &
    (df["end_station_name"].isin(["Canal St at Causeway St",
    "Canal St\xa0at\xa0Causeway\xa0St"])), "end_station_name"] = standard_name

# Final check for mismatch
validate_station_mappings(df)

Start Station Names Mapping to Multiple IDs:
nil

Start Station IDs Mapping to Multiple Names:
nil

End Station Names Mapping to Multiple IDs:
nil

End Station IDs Mapping to Multiple Names:
nil


In [ ]:
# Save the cleaned dataset for reuse
cleaned_path = base_dir + "cleaned_trips.csv"
df.to_csv(cleaned_path, index=False)
print(f"Cleaned data saved")

Cleaned data saved to: C:/Users/natda/Desktop/NatDave/Academics/PhD_NU/RESEARCH/BlueBikes/cleaned_trips.csv
